In [1]:
import skbio
from Bio import SeqIO
import pandas as pd
workdir = "/Volumes/McMinds/git_repos/gcmp_stan_symportal_2019/"
outdir = workdir + "output/symbio_phylo/"

In [3]:
input_file = "all_seqs.fasta"
#input_file = "72_DBV_20190609_2019-06-10_01-30-10.537772.seqs.fasta"
discard_unnamed = True
clades = {}
for seq in SeqIO.parse(workdir + "raw_data/symbio_phylo/" + input_file, "fasta"):
        
        if "_" in seq.id:
            if discard_unnamed:
                continue
            else:
                clade = seq.id.split("_")[1]
        else:
            clade = seq.id[0]
            
        if clade not in clades.keys():
            clades[clade] = []
            
        clades[clade].append(seq)   
        
clade_sizes = {}
for x in clades.keys():
    clade_sizes[x] = len(clades[x])
    print(x + " " + str(clade_sizes[x]))
    
clades_sorted = sorted(clade_sizes, key=clade_sizes.get)

A 247
B 76
C 706
F 55
D 119
E 1
G 28
H 9
I 4


In [ ]:
for x in clades_sorted:
    prefix = outdir + "its2_clade_" + x
    nseqs = len(clades[x])

    print('Writing ' + str(nseqs) + ' clade ' + x + ' seq(s) to a file\n')
    SeqIO.write(clades[x], prefix + ".fasta", "fasta-2line")
    
    if nseqs > 1:
        print('Aligning clade ' + x + ':')
        !mafft --thread 4 --maxiterate 1000 --ep 0 --genafpair {prefix + ".fasta"} > {prefix + "_aligned.fasta"}
        
        #print('Building phylogeny for clade ' + x + ':')
        #!iqtree -s {prefix + "_aligned.fasta"}

In [ ]:
seeds = ""
addseqs = "/dev/null"
inds = []
for x in clades_sorted:
    if len(clades[x]) > 1:
        seeds = seeds + "--seed " + outdir + "its2_clade_" + x + "_aligned.fasta "
    else:
        inds.append(clades[x])

if len(inds) > 0:
    if len(inds) == 1:
        inds = inds[0]
    print('Writing isolate clades to a file\n')
    addseqs = outdir + "its2_concat_isolates.fasta"
    SeqIO.write(inds, addseqs, "fasta-2line")       
        
!mafft --thread 4 --maxiterate 1000 --ep 0 --genafpair {seeds}{addseqs} > {outdir + "its2_all_clades_aligned.fasta"}

In [ ]:
partitions_file = outdir + "iqtree_partitions.nex"
f = open(partitions_file, "w+")

part = 1
f.write("#nexus\n")
f.write("begin sets;\n")
f.write("    charset ITS2 = " + outdir + "its2_all_clades_aligned_clean.fasta:*;\n")
for gene in ("28S","23S","cob","coi","elf","psba"):
    part += 1
    f.write("    charset " + gene + " = " + workdir + "raw_data/symbio_phylo/" + gene + "/forindividual tree " + gene + ".fas:*;\n")
f.write("end;\n")
f.close()

In [ ]:
!sed 's/_seed_//g' {outdir + "its2_all_clades_aligned.fasta"} > {outdir + "its2_all_clades_aligned_clean.fasta"}
!iqtree -nt AUTO -ntmax 4 -pre {outdir + "allgenes"} -mrate E,I,G,I+G,R,H,I+H -mfreq F,FO -spp {partitions_file}

In [4]:
abundance = workdir + "raw_data/symbio_phylo/72_DBV_20190609_2019-06-10_01-30-10.537772.profiles.absolute.txt"
df = pd.read_csv(abundance, sep="\t", index_col=0) 
df.drop(columns=[df.columns[0]], inplace=True)

In [5]:
profs = dict(df.loc['ITS2 type profile'])

In [6]:
from ete3 import Tree
tree = Tree(outdir + "allgenes.treefile")

alldivs = set()
allnames = {}
profile_map_file = outdir + "profile_map.txt"

f = open(profile_map_file, "w+")

for prof in profs.keys():
    f.write(prof + "_*" + " " + prof + '\n')
    divs = profs[prof].replace('/',',').replace('-',',')
    for div in divs.split(','):
        newname = prof + "_" + div
        alldivs.add(newname)
        if div not in allnames.keys():
            allnames[div] = [newname]
            tree.search_nodes(name=div)[0].name = newname
        else:
            node = tree.search_nodes(name=allnames[div][0])[0]
            if node.up.name == div:
                node.up.add_child(name=newname, dist=0)
            else:
                node.name = div
                node.add_child(name=allnames[div][0], dist=0)
                node.add_child(name=newname, dist=0)
            allnames[div].append(newname)
f.close()

tree.prune(alldivs, preserve_branch_length=True)
gene_trees_folder = outdir + "its2tree/"
pruned_tree = gene_trees_folder + "allgenes.pruned.treefile"
!mkdir {gene_trees_folder}
tree.write(format=5, outfile=pruned_tree)

mkdir: /Volumes/McMinds/git_repos/gcmp_stan_symportal_2019/output/symbio_phylo/its2tree/: File exists


In [27]:
!rm -rf {outdir + "STAG_Results*"}
!python2 /Applications/STAG-1.0.0/stag/stag.py {profile_map_file} {gene_trees_folder}
!cp -r {outdir + "STAG_Results?*"} {outdir + "STAG_Results"}
!rm -rf {outdir + "STAG_Results?*"}
profTree = skbio.TreeNode.read(outdir + "STAG_Results/SpeciesTree.tre").root_at_midpoint()
profTree.write(file = outdir + "STAG_Results/SpeciesTree_rooted.tree")
model = 'correlated'
collapseMultis = True
!Rscript {workdir + 'gcmp_stan_symportal/phylogenetics/chronogram.r'} {collapseMultis} {model} {outdir + "STAG_Results/SpeciesTree_rooted.tree"} {outdir + "STAG_Results/SpeciesTree_" + model + "_chronos.tree"}

*********************************************************
*                                                       *
*      STAG: Species Tree inference from All Genes      *
*                                                       *
*********************************************************

274 species in mapping file:
34124
34127
34126
34121
34418
34123
34122
34486
34414
34251
34416
34129
34410
34659
34494
34398
34712
34559
34560
34393
34390
34557
34120
34091
34651
34652
34524
34301
34518
34152
34153
34154
34155
34156
34157
34158
34159
34463
34417
34467
34136
34137
34134
34135
34644
34133
34408
34409
34406
34138
34402
34400
34401
34529
34722
34484
34720
34480
34481
34520
34283
34522
34304
34172
34449
34086
34717
34382
34423
34314
34558
34207
34490
34102
34438
34433
34432
34437
34435
34434
34637
34636
34630
34633
34532
34530
34531
34537
34534
34274
34036
34035
34034
34033
34032
34031
34030
34395
34039
34506
34117
34349
34348
34114
34347
34220
34110
34058
34509
34113
34626
34217
34620
34

In [7]:
import re
abunds = {}
for string,profdef,names in zip(df.loc['Average defining sequence proportions and [stdev]'], df.loc['ITS2 type profile'], df.columns):
    abunds[names] = {}
    for substr,div in zip(string.split('-'),profdef.replace('/','-').split('-')):
        abunds[names][div] = float(re.sub('\[.*\]','',substr)) * 1000 # need integer values for some reason! provided decimal values were at precision of 1e-3
abunds = pd.DataFrame.from_dict(abunds, orient='index')
abunds[pd.isna(abunds)] = 0

In [8]:
divTree = skbio.TreeNode.read(outdir + "allgenes.treefile").root_at_midpoint()
divTree.write(file = outdir + "allgenes_rooted.treefile")

'/Volumes/McMinds/git_repos/gcmp_stan_symportal_2019/output/symbio_phylo/allgenes_rooted.treefile'

In [ ]:
seqdf = pd.read_csv(outdir + "allgenes.uniqueseq.phy", sep="\s+", index_col=0, header=None)
f = open(outdir + "allgenes.uniqueseq.replicated.phy", 'w')
f.write(str(len(alldivs)) + ' ')
with open(outdir + "allgenes.uniqueseq.phy") as f2:
    f.write(next(f2).split(' ')[1])
for div in allnames.keys():
    for name in allnames[div]:
        newname = name.split('_')[0]
        newname = newname + ' ' * (10 - len(newname))
        f.write(newname + list(seqdf.loc[div])[0] + '\n')
f.close()

In [ ]:
!iqtree -mfreq F,FO -mrate E,I,G,I+G,R,H,I+H -s {outdir + "its2_all_clades_aligned.fasta"}

In [ ]:
from ete3 import Tree
gene_trees_folder = outdir + "its2tree/"
pruned_tree = gene_trees_folder + "its2_all_clades_aligned.fasta.pruned.treefile"
!mkdir {gene_trees_folder}
!sed 's/_seed_//g' {outdir + "its2_all_clades_aligned.fasta.treefile"} > {pruned_tree}
tree = Tree(pruned_tree)
tree.prune(alldivs, preserve_branch_length=True)
tree.write(format=1, outfile=pruned_tree)

In [ ]:
for gene in ("28S","23S","cob","coi","elf","psba"):
    !iqtree -pre {outdir + "forindividual_tree_"+gene+"_"} -mtree -mrate E,I,G,I+G,R,H,I+H -mfreq F,FO -s {workdir + "raw_data/symbio_phylo/"+gene+"/forindividual\ tree\ "+gene+".fas"}

In [ ]:
import ete3
leaves = []
for leaf in tree:
    leaves.append(leaf.name)

In [ ]:
for prof in profs.keys():
    divs = profs[prof].replace('/',',').replace('-',',') 
    if not set(divs.split(',')).issubset(leaves):
        print(profs[prof])

In [ ]:
treeS = Tree("/Volumes/McMinds/git_repos/gcmp_stan_symportal_2019/output/symbio_phylo/STAG_ResultsJun26/SpeciesTree.tree")
newleaves = []
for leaf in treeS:
    newleaves.append(leaf.name)
for prof in profs.keys():
    if prof not in newleaves:
        print(profs[prof])

In [ ]:
treeS = Tree("/Volumes/McMinds/git_repos/gcmp_stan_symportal_2019/output/symbio_phylo/STAG_ResultsJun26/SpeciesTree.tre")
newleaves = []
for leaf in treeS:
    newleaves.append(leaf.name)
for prof in profs.keys():
    if prof in newleaves:
        print(profs[prof])